# 👻 GHOST Face Swapping — Google Colab Notebook

This notebook sets up the GHOST face swapping pipeline on Google Colab.
It follows the CPU/GPU-compatible code path from the latest repository version.


## ⚠️ Ethical Use Only
Use this software responsibly and with consent from all parties involved.
Do not generate or share deceptive or harmful content.


## 1️⃣ Check GPU & Python Runtime
Verify that Google Colab is running with GPU acceleration (Runtime → Change runtime type → GPU).


In [ ]:
!nvidia-smi || echo '⚠️ GPU not available. Falling back to CPU.'
import sys
print(f"Python version: {sys.version}")


## 2️⃣ Clone the Repository
Update the `REPO_URL` if you are testing a fork or a specific branch.


In [ ]:
import os
REPO_URL = "https://github.com/idrismusa4/ghost.git"  # Replace with your fork if needed
BRANCH = "main"  # Set to the branch or tag you want to test
PROJECT_DIR = "/content/ghost"

if os.path.exists(PROJECT_DIR):
    !rm -rf "{PROJECT_DIR}"

!git clone --depth 1 --branch "$BRANCH" "$REPO_URL" "$PROJECT_DIR"
%cd "$PROJECT_DIR"


## 3️⃣ Install Dependencies
PyTorch wheels with CUDA support are installed explicitly; the rest come from `requirements.txt`.


In [ ]:
!pip install -q --upgrade pip
!pip install -q --extra-index-url https://download.pytorch.org/whl/cu118     torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2
!pip install -q -r requirements.txt


## 4️⃣ Download Pretrained Weights
Run the helper script to fetch ArcFace, landmark detectors, generator/discriminator weights, and the super-resolution model.


In [ ]:
!bash download_models.sh


## 5️⃣ Prepare Your Inputs
Upload a **source** face image and a **target** image or video.
Files are saved under the `input/` directory and their paths are recorded for later steps.


In [ ]:
from google.colab import files
import os

os.makedirs('input', exist_ok=True)

print('📤 Upload the SOURCE face image:')
source_upload = files.upload()
source_name = next(iter(source_upload))
source_ext = os.path.splitext(source_name)[1] or '.jpg'
source_path = os.path.join('input', f'source{source_ext}')
os.replace(source_name, source_path)

print('
📤 Upload the TARGET image or video:')
target_upload = files.upload()
target_name = next(iter(target_upload))
target_ext = os.path.splitext(target_name)[1] or '.jpg'
target_path = os.path.join('input', f'target{target_ext}')
os.replace(target_name, target_path)

with open('source_path.txt', 'w') as f:
    f.write(source_path)
with open('target_path.txt', 'w') as f:
    f.write(target_path)

print(f'
✅ Source saved to: {source_path}')
print(f'✅ Target saved to: {target_path}')


## 6️⃣ Run Image-to-Image Face Swap
This cell runs only when the uploaded target is an image.
`--device cuda` will use the GPU; switch to `cpu` if needed.


In [ ]:
import os, subprocess, shlex

with open('source_path.txt') as f:
    source_path = f.read().strip()
with open('target_path.txt') as f:
    target_path = f.read().strip()

image_exts = {'.png', '.jpg', '.jpeg', '.bmp'}
if os.path.splitext(target_path)[1].lower() in image_exts:
    os.makedirs('output', exist_ok=True)
    cmd = [
        'python', 'inference.py',
        '--image_to_image', 'True',
        '--target_image', target_path,
        '--source_paths', source_path,
        '--out_image_name', 'output/result_image.png',
        '--device', 'cuda',
        '--batch_size', '4'
    ]
    print('Running command:', ' '.join(shlex.quote(part) for part in cmd))
    subprocess.run(cmd, check=True)
else:
    print('ℹ️ Target file is not an image. Skip this cell or upload an image target.')


## 7️⃣ Run Image-to-Video Face Swap
This cell triggers when the target is a video file. Adjust `--frame_processor` arguments if you want masking or enhancement.


In [ ]:
import os, subprocess, shlex

with open('source_path.txt') as f:
    source_path = f.read().strip()
with open('target_path.txt') as f:
    target_path = f.read().strip()

video_exts = {'.mp4', '.mov', '.mkv', '.avi', '.webm'}
if os.path.splitext(target_path)[1].lower() in video_exts:
    os.makedirs('output', exist_ok=True)
    cmd = [
        'python', 'inference.py',
        '--image_to_image', 'False',
        '--target_video', target_path,
        '--source_paths', source_path,
        '--output_path', 'output/result_video.mp4',
        '--device', 'cuda',
        '--batch_size', '4',
        '--keep_audio', 'True'
    ]
    print('Running command:', ' '.join(shlex.quote(part) for part in cmd))
    subprocess.run(cmd, check=True)
else:
    print('ℹ️ Target file is not a recognised video. Skip this cell or upload a video target.')


## 8️⃣ Preview & Download Results
List generated outputs, display images inline, and preview MP4 videos directly in Colab.


In [ ]:
import os
from IPython.display import Image, display, HTML
from base64 import b64encode

if not os.path.exists('output'):
    print('No output directory found yet. Run the inference cells first.')
else:
    files = os.listdir('output')
    if not files:
        print('The output directory is empty.')
    else:
        print('Output directory contents:')
        for name in files:
            print('-', name)
        for name in files:
            path = os.path.join('output', name)
            if name.lower().endswith((".png", ".jpg", ".jpeg")):
                print(f"\n🖼 Displaying {name}")
                display(Image(filename=path))
            elif name.lower().endswith('.mp4'):
                print(f"\n🎬 Previewing {name}")
                data_url = "data:video/mp4;base64," + b64encode(open(path, 'rb').read()).decode()
                display(HTML(f"<video width=640 controls><source src='{data_url}' type='video/mp4'></video>"))


## 9️⃣ (Optional) Download Outputs Locally
Run this cell to download generated files to your machine.


In [ ]:
import os
from google.colab import files

if os.path.exists('output'):
    for name in os.listdir('output'):
        path = os.path.join('output', name)
        if os.path.isfile(path):
            files.download(path)
else:
    print('No output directory found yet.')
